In [274]:
import json
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
from pvd_io import *

### Load Pairwise Segment Data

In [275]:
data_dir = 'pvd_analysis/'
min_file_size = 1e3  # 1 kb
datasets, sessions, files = scan_directories(data_dir, min_file_size, filetype='voxels.pkl')
print(f"Located {len(files)} matched segment lists.")

Located 47 matched segment lists.


In [276]:
# Load Pickled lists of change matrices
matched_segments_voxels = []
matched_segments_spline = []

for ii, session in enumerate(sessions):

    file_path_vx = f"{data_dir}/{datasets[ii]}/{session}/length_change_voxels.pkl"
    file_path_sp = f"{data_dir}/{datasets[ii]}/{session}/length_change_spline.pkl"

    with open(file_path_vx, 'rb') as file:
        session_segments_vx = pickle.load(file)
    with open(file_path_sp, 'rb') as file:
        session_segments_sp = pickle.load(file)

    matched_segments_voxels.append(session_segments_vx)
    matched_segments_spline.append(session_segments_sp)

### Create Dataframe

In [277]:
# Take all sessions and extract pairwise changes to a pandas df
seg_changes = []
seg_dataset = []
seg_session = []
seg_timept = []
seg_id = []

# Calculate pairwise changes and populate related lists for columns
for ii, matched_segments in enumerate(matched_segments_voxels):
    for kk, segment in enumerate(matched_segments):
        for vv in range(segment.shape[0]-1):
            change = segment[vv+1, vv]
            seg_changes.append(change)
            seg_dataset.append(datasets[ii])
            seg_session.append(sessions[ii])
            seg_timept.append(vv+1)
            seg_id.append(kk)

# Create pandas dataframe from lists
seg_change_df = pd.DataFrame({'pairwise_change': seg_changes, 'timepoint': seg_timept, 'segment_id': seg_id, 'dataset': seg_dataset, 'session': seg_session})

# Remove underscores from session directory names, if needed.
def remove_trailing_underscore(value):
    if isinstance(value, str) and value.endswith('_'):
        return value[:-1]
    return value

seg_change_df = seg_change_df.map(remove_trailing_underscore)

# Load conditions key and map to session values
with open('session_conditions.json', 'rb') as f:
    session_conditions = json.load(f)

seg_change_df['condition'] = seg_change_df['session'].map(session_conditions)

In [278]:
# Preview the dataframe
seg_change_df.loc[seg_change_df['session'] == 'expDS4_35']

,pairwise_change,timepoint,segment_id,dataset,session,condition
3024,-4,1,0,DataSet04,expDS4_35,Control
3025,1,2,0,DataSet04,expDS4_35,Control
3026,-4,3,0,DataSet04,expDS4_35,Control
3027,2,1,1,DataSet04,expDS4_35,Control
3028,-2,2,1,DataSet04,expDS4_35,Control
...,...,...,...,...,...,...
3115,4,2,30,DataSet04,expDS4_35,Control
3116,0,3,30,DataSet04,expDS4_35,Control
3117,0,1,31,DataSet04,expDS4_35,Control
3118,-3,2,31,DataSet04,expDS4_35,Control


In [279]:
# Preview a single segment
seg_change_df.loc[(seg_change_df['session'] == 'expDS4_35') & (seg_change_df['session'])]

,pairwise_change,timepoint,segment_id,dataset,session,condition
3024,-4,1,0,DataSet04,expDS4_35,Control
3025,1,2,0,DataSet04,expDS4_35,Control
3026,-4,3,0,DataSet04,expDS4_35,Control
3027,2,1,1,DataSet04,expDS4_35,Control
3028,-2,2,1,DataSet04,expDS4_35,Control
...,...,...,...,...,...,...
3115,4,2,30,DataSet04,expDS4_35,Control
3116,0,3,30,DataSet04,expDS4_35,Control
3117,0,1,31,DataSet04,expDS4_35,Control
3118,-3,2,31,DataSet04,expDS4_35,Control


In [280]:
# Aggressively remove outliers as a test
#seg_change_df = seg_change_df[(seg_change_df['pairwise_change'] < 30) & (seg_change_df['pairwise_change'] > -30)]

In [281]:
# Identify some outliers for visual inspection
outlier_df = seg_change_df[(seg_change_df['pairwise_change'] > 30) | (seg_change_df['pairwise_change'] < -30)]
outlier_df

,pairwise_change,timepoint,segment_id,dataset,session,condition
170,-31,3,21,DataSet00,exp240129_01_01,Control
196,33,2,30,DataSet00,exp240129_01_01,Control
316,33,2,13,DataSet00,exp240104_01_03,DOI
506,35,3,17,DataSet00,exp240104_00_02,Control
1118,-31,3,1,DataSet01,exp240202_00_E,Control
1335,-31,1,28,DataSet01,exp240202_00_G,Control
1359,-42,1,7,DataSet04,expDS4_31,DOI
1461,36,1,14,DataSet04,expDS4_06,DOI
1576,33,2,13,DataSet04,expDS4_14,DOI
1590,-31,1,18,DataSet04,expDS4_14,DOI


In [282]:
# Plot absolutely everything
fig = px.histogram(seg_change_df, x='pairwise_change', histnorm='probability density')

# Get the mean
mean_value = np.mean(seg_change_df['pairwise_change'])
std_value = np.std(seg_change_df['pairwise_change'])

# Add a vertical line for the mean
fig.add_vline(x=mean_value, 
              line_width=1, 
              line_dash="dash", 
              line_color="black", 
              annotation_text=f"{mean_value:.2f}", 
              annotation_position="top")

fig.add_vline(x=std_value*3, 
              line_width=1, 
              line_dash="dash", 
              line_color="red", 
              annotation_text=f"{std_value:.2f}", 
              annotation_position="top")

fig.add_vline(x=std_value*-3, 
              line_width=1, 
              line_dash="dash", 
              line_color="red", 
              annotation_text=f"{std_value:.2f}", 
              annotation_position="top")

fig.show()

In [283]:
def pairwise_change_histogram(dataset, condition, nbins=None, pdf=True):
    # Filter dataframe
    drug_df = seg_change_df[(seg_change_df['condition'] == condition ) & (seg_change_df['dataset'] == dataset)]
    # Make histogram
    if pdf:
        fig = px.histogram(drug_df, x='pairwise_change', histnorm='probability density', nbins=nbins)
    else:
        fig = px.histogram(drug_df, x='pairwise_change')
    # Get the mean
    mean_value = np.mean(drug_df['pairwise_change'])
    std_value = np.std(drug_df['pairwise_change'])
    # Add a vertical line for the mean
    fig.add_vline(x=mean_value, line_width=1, line_dash="dash", line_color="black", annotation_text=f"{mean_value:.2f}", annotation_position="top")
    # Add a vertical lines for 3 sigma
    multipliers = [3,-3]
    for mult in multipliers:
        fig.add_vline(x=std_value*mult, line_width=1, line_dash="dash", line_color="red", annotation_text=f"", annotation_position="top")

    fig.update_layout(title_text=f"{dataset}<br>Condition: <b>{condition}</b><br>")

    fig.show()

In [284]:
dataset = 'DataSet00'
pairwise_change_histogram(dataset=dataset, condition='Control')
pairwise_change_histogram(dataset=dataset, condition='DOI')

In [285]:
dataset = 'DataSet01'
pairwise_change_histogram(dataset=dataset, condition='Control')
pairwise_change_histogram(dataset=dataset, condition='DOI')

In [286]:
dataset = 'DataSet04'
pairwise_change_histogram(dataset=dataset, condition='Control')
pairwise_change_histogram(dataset=dataset, condition='DOI')

### Statistics

In [287]:
from scipy import stats

dataset = 'DataSet04'
control_data = seg_change_df[(seg_change_df['condition'] == 'Control' ) & (seg_change_df['dataset'] == dataset)]
control_data = control_data['pairwise_change']
doi_data = seg_change_df[(seg_change_df['condition'] == 'DOI' ) & (seg_change_df['dataset'] == dataset)]
doi_data = doi_data['pairwise_change']

In [288]:
# Perform Levene's test
statistic, p_value = stats.levene(control_data, doi_data)
print(f"Levene's test statistic: {statistic}")
print(f"p-value: {p_value}")

# Interpret the results
alpha = 0.05  # common significance level
if p_value > alpha:
    print("Fail to reject the null hypothesis. The variances are likely equal.")
else:
    print("Reject the null hypothesis. The variances are likely not equal.")

Levene's test statistic: 0.026939734169747
p-value: 0.8696442197366224
Fail to reject the null hypothesis. The variances are likely equal.


In [289]:
def bootstrap_mean_diff(group1, group2, num_bootstraps=10000):
    n1, n2 = len(group1), len(group2)
    combined = np.concatenate([group1, group2])
    bootstrap_diffs = []
    for _ in range(num_bootstraps):
        boot_sample = np.random.choice(combined, size=n1+n2, replace=True)
        boot_group1 = boot_sample[:n1]
        boot_group2 = boot_sample[n1:]
        bootstrap_diffs.append(np.mean(boot_group1) - np.mean(boot_group2))
    return bootstrap_diffs

bootstrap_diffs = bootstrap_mean_diff(control_data, doi_data)
observed_diff = np.mean(control_data) - np.mean(doi_data)

# Calculate p-value
p_value = np.mean(np.abs(bootstrap_diffs) >= np.abs(observed_diff))

print(f"Observed difference in means: {observed_diff}")
print(f"Bootstrap p-value: {p_value}")

# Calculate confidence interval
ci_lower, ci_upper = np.percentile(bootstrap_diffs, [2.5, 97.5])
print(f"95% Confidence Interval: ({ci_lower}, {ci_upper})")

Observed difference in means: -0.0935077519379845
Bootstrap p-value: 0.8111
95% Confidence Interval: (-0.7663401162790698, 0.7570038759689921)
